## Step 1: Set Up Imports

In [36]:
import pandas as pd
import plotly.express as px
import numpy as np

## Step 1b: Import the data (issue with encoder)

In [37]:
#data had an encoding issue. Byte was not compatible with UTF-8
#cp1252 was compatible for decoding 0x96 byte

df = pd.read_csv('/Users/kali/Documents/GitHub/Taylor_swift/taylor.csv', encoding='cp1252')
df.head()

,City,Country,Venue,Opening act(s),Attendance (tickets sold / available),Revenue,Tour
0,Evansville,United States,Roberts Municipal Stadium,Gloriana\r\nKellie Pickler,"7,463 / 7,463","$360,617",Fearless_Tour
1,Jonesboro,United States,Convocation Center,Gloriana\r\nKellie Pickler,"7,822 / 7,822","$340,328",Fearless_Tour
2,St. Louis,United States,Scottrade Center,Gloriana\r\nKellie Pickler,"13,764 / 13,764","$650,420",Fearless_Tour
3,Alexandria,United States,Bishop Ireton High School,Gloriana\r\nKellie Pickler,—,—,Fearless_Tour
4,North Charleston,United States,North Charleston Coliseum,Gloriana\r\nKellie Pickler,"8,751 / 8,751","$398,154",Fearless_Tour


## Step 2: Tidy Data

In [38]:
#Pep 8 compliance: Column headers be snake cased. 

df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
df.columns

Index(['city', 'country', 'venue', 'opening_act(s)',
       'attendance_(tickets_sold_/_available)', 'revenue', 'tour'],
      dtype='object')

## Step 3:EDA(Exploratory Data Analysis)

To gather summary statistics about our dataset we can use the following methods to identify if any further tidying is necessary.

* df.describe()

* df.info()

* df.shape

* df.value_counts()

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   city                                   445 non-null    object
 1   country                                445 non-null    object
 2   venue                                  445 non-null    object
 3   opening_act(s)                         444 non-null    object
 4   attendance_(tickets_sold_/_available)  442 non-null    object
 5   revenue                                442 non-null    object
 6   tour                                   445 non-null    object
dtypes: object(7)
memory usage: 24.5+ KB


Clean revenue column name

In [40]:
df['revenue'] = df['revenue'].str.replace('-', '').str.replace('$', '').str.replace(',', '')

change revenue to float data type

In [41]:
#df["revenue"] = df["revenue"].astype(float, errors= 'coerce')

df["revenue"] = pd.to_numeric(df["revenue"], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   445 non-null    object 
 1   country                                445 non-null    object 
 2   venue                                  445 non-null    object 
 3   opening_act(s)                         444 non-null    object 
 4   attendance_(tickets_sold_/_available)  442 non-null    object 
 5   revenue                                406 non-null    float64
 6   tour                                   445 non-null    object 
dtypes: float64(1), object(6)
memory usage: 24.5+ KB


fill in revenue null values

In [42]:
df["revenue"].fillna(df["revenue"].mean(), inplace=True)

check revenue filled in Null Values

In [43]:
df.isna().sum()

city                                     0
country                                  0
venue                                    0
opening_act(s)                           1
attendance_(tickets_sold_/_available)    3
revenue                                  0
tour                                     0
dtype: int64

cleaning column name for attendance_etc,

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   445 non-null    object 
 1   country                                445 non-null    object 
 2   venue                                  445 non-null    object 
 3   opening_act(s)                         444 non-null    object 
 4   attendance_(tickets_sold_/_available)  442 non-null    object 
 5   revenue                                445 non-null    float64
 6   tour                                   445 non-null    object 
dtypes: float64(1), object(6)
memory usage: 24.5+ KB


First have to create two new columns to get rid of the slash

In [45]:
df['tick_sold'] = df['attendance_(tickets_sold_/_available)'].str.split(" / ").str[0].str.replace(',', '')
df['tick_avail'] = df['attendance_(tickets_sold_/_available)'].str.split(" / ").str[1].str.replace(',', '')
df.head()

,city,country,venue,opening_act(s),attendance_(tickets_sold_/_available),revenue,tour,tick_sold,tick_avail
0,Evansville,United States,Roberts Municipal Stadium,Gloriana\r\nKellie Pickler,"7,463 / 7,463",3.606170e+05,Fearless_Tour,7463,7463
1,Jonesboro,United States,Convocation Center,Gloriana\r\nKellie Pickler,"7,822 / 7,822",3.403280e+05,Fearless_Tour,7822,7822
2,St. Louis,United States,Scottrade Center,Gloriana\r\nKellie Pickler,"13,764 / 13,764",6.504200e+05,Fearless_Tour,13764,13764
3,Alexandria,United States,Bishop Ireton High School,Gloriana\r\nKellie Pickler,—,3.892357e+06,Fearless_Tour,—,NaN
4,North Charleston,United States,North Charleston Coliseum,Gloriana\r\nKellie Pickler,"8,751 / 8,751",3.981540e+05,Fearless_Tour,8751,8751


In [46]:
df["tick_sold"] = pd.to_numeric(df["tick_sold"], errors='coerce')
df["tick_avail"] = pd.to_numeric(df["tick_avail"], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 9 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   445 non-null    object 
 1   country                                445 non-null    object 
 2   venue                                  445 non-null    object 
 3   opening_act(s)                         444 non-null    object 
 4   attendance_(tickets_sold_/_available)  442 non-null    object 
 5   revenue                                445 non-null    float64
 6   tour                                   445 non-null    object 
 7   tick_sold                              407 non-null    float64
 8   tick_avail                             407 non-null    float64
dtypes: float64(3), object(6)
memory usage: 31.4+ KB


In [47]:
df["tick_sold"].fillna(df["tick_sold"].mean(), inplace=True)
df["tick_avail"].fillna(df["tick_avail"].mean(), inplace=True)
df.head()

,city,country,venue,opening_act(s),attendance_(tickets_sold_/_available),revenue,tour,tick_sold,tick_avail
0,Evansville,United States,Roberts Municipal Stadium,Gloriana\r\nKellie Pickler,"7,463 / 7,463",3.606170e+05,Fearless_Tour,7463.000000,7463.000000
1,Jonesboro,United States,Convocation Center,Gloriana\r\nKellie Pickler,"7,822 / 7,822",3.403280e+05,Fearless_Tour,7822.000000,7822.000000
2,St. Louis,United States,Scottrade Center,Gloriana\r\nKellie Pickler,"13,764 / 13,764",6.504200e+05,Fearless_Tour,13764.000000,13764.000000
3,Alexandria,United States,Bishop Ireton High School,Gloriana\r\nKellie Pickler,—,3.892357e+06,Fearless_Tour,38954.904177,39002.093366
4,North Charleston,United States,North Charleston Coliseum,Gloriana\r\nKellie Pickler,"8,751 / 8,751",3.981540e+05,Fearless_Tour,8751.000000,8751.000000


dropping the attendance_etc column

In [48]:
df.drop(columns='attendance_(tickets_sold_/_available)', inplace=True)
df.head()

,city,country,venue,opening_act(s),revenue,tour,tick_sold,tick_avail
0,Evansville,United States,Roberts Municipal Stadium,Gloriana\r\nKellie Pickler,3.606170e+05,Fearless_Tour,7463.000000,7463.000000
1,Jonesboro,United States,Convocation Center,Gloriana\r\nKellie Pickler,3.403280e+05,Fearless_Tour,7822.000000,7822.000000
2,St. Louis,United States,Scottrade Center,Gloriana\r\nKellie Pickler,6.504200e+05,Fearless_Tour,13764.000000,13764.000000
3,Alexandria,United States,Bishop Ireton High School,Gloriana\r\nKellie Pickler,3.892357e+06,Fearless_Tour,38954.904177,39002.093366
4,North Charleston,United States,North Charleston Coliseum,Gloriana\r\nKellie Pickler,3.981540e+05,Fearless_Tour,8751.000000,8751.000000


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            445 non-null    object 
 1   country         445 non-null    object 
 2   venue           445 non-null    object 
 3   opening_act(s)  444 non-null    object 
 4   revenue         445 non-null    float64
 5   tour            445 non-null    object 
 6   tick_sold       445 non-null    float64
 7   tick_avail      445 non-null    float64
dtypes: float64(3), object(5)
memory usage: 27.9+ KB


In [50]:
df.isna().sum()

city              0
country           0
venue             0
opening_act(s)    1
revenue           0
tour              0
tick_sold         0
tick_avail        0
dtype: int64

In [51]:

df['opening_act(s)'] = df['opening_act(s)'].fillna('unavailable')

In [52]:
df.isna().sum()

city              0
country           0
venue             0
opening_act(s)    0
revenue           0
tour              0
tick_sold         0
tick_avail        0
dtype: int64

# Create visualizations

In [53]:
px.histogram(df, 'revenue')

In [54]:
px.histogram(df, 'opening_act(s)')

In [55]:
for col in df.columns:
    if df[col].dtype != '0':
        display(px.histogram(df,col))

In [56]:
df.to_csv('new_clean_taylor.csv', encoding='cp1252')

